# Welcome to the Notebook for my final submission

##### You will notice that I supressed the part of the code that contains my API access data, for security reasons.  
##### Enjoy the rest of the code! :D

In [34]:
#importing libraries
import pandas as pd
import numpy as np
import warnings
import urllib.request
import requests # library to handle requests

from geopy.geocoders import Nominatim
from bs4 import BeautifulSoup
from pandas.io.json import json_normalize

warnings.filterwarnings('ignore')

##### So, for starters, let's craft our database

In [21]:
#First, let's get a list of Florianópolis'neighborhoods! 
url = 'https://pt.wikipedia.org/wiki/Lista_de_distritos_e_bairros_de_Florian%C3%B3polis'
req = urllib.request.urlopen(url)
article = req.read().decode()

with open('neighb.html', 'w') as code:
    code.write(article)

In [22]:
article = open('neighb.html').read()
soup = BeautifulSoup(article, 'html.parser')
#get that table
tables = soup.find_all('table', class_='sortable')

In [5]:
# Search through the tables for the one with the headings we want.
# The wikipedia page has two tables: one for districts and one for neighborhood. 
# We are interested in the one that contains the neighborhoods
for table in tables:
    ths = table.find_all('th')
    headings = [th.text.strip() for th in ths]
    if headings[:5] == ['Posição', 'Bairro', 'População']:
        break

##### Please note that in the following section we will perform the table scraping while obtaining the geospatial data through an API !

In [15]:
key = '54002a88657b471f8443056614be2e37'

In [18]:
from opencage.geocoder import OpenCageGeocode
# Extract the columns we want and write to a colon-delimited text file.
with open('pop_city.csv', 'w') as pop:
    print('position,', 'neighborhood,', 'population,', 'latitude,', 'longitude', file = pop)
    for tr in table.find_all('tr'):
        tds = tr.find_all('td')
        if (not tds) or (len(tds)<3) :
            continue
        position, neighborhood, population = [td.text.strip() for td in tds[:3]]

        #The following bit is about the API query:
        geocoder = OpenCageGeocode(key)
        query = neighborhood + ', Florianopolis, Brazil'
        results = geocoder.geocode(query)

        latitude = str(results[0]['geometry']['lat'])
        longitude = str(results[0]['geometry']['lng']) 
        print(', '.join([position, neighborhood, population, latitude, longitude]), file=pop)

##### Let us open the .csv file and see if it turned out ok:

In [19]:
df_pop = pd.read_csv('pop_city.csv', sep=r'\s*,\s*')
df_pop.head()

,position,neighborhood,population,latitude,longitude
0,1,Centro,44.074,-27.597203,-48.549481
1,2,Capoeiras,19.323,-27.597333,-48.590008
2,3,Trindade,15.031,-27.589383,-48.522400
3,4,Agronômica,14.591,-27.578145,-48.535717
4,5,Saco dos Limões,13.771,-27.608268,-48.534343


In [20]:
df_pop.tail()

,position,neighborhood,population,latitude,longitude
80,81,Forte,266.0,-27.421111,-48.432848
81,82,Ingleses Norte,203.0,-27.418779,-48.408051
82,83,Caieira,170.0,-27.596691,-48.535275
83,84,Praia Brava,130.0,-27.397613,-48.415825
84,85,Praia Mole,108.0,-27.603017,-48.433248


##### Seems ok to me!

### Now, let's get the coordinates for the universities in Florianópolis
As it happens, the top rated universities in the city (according to the Brazilian index RUF) are:

* UFSC - Federal University of Santa Catarina  
* IFSC - Federal Institude of Santa Catarina  
* UDESC - University of the State of Santa Cartarina  
* UNIVALI - University of the Itajaí Valley  
* UNISUL - University of the South of Santa Catarina  

So, let's obtain their coordinates, shall we?

In [ ]:
universities = pd.DataFrame(columns = ['University', 'Latitude', 'Longitude'])

In [28]:
#Firing up that Foursquare API
CLIENT_ID = '00RRGR34RRRRSAZBIIEHKUQSDC0OIHWY0X3LKNSWA3CBCMDY' # your Foursquare ID
CLIENT_SECRET = 'VQEO5QPVRG0KORW4HJKNOAMMQQZVRGBQAK145E30WKCRKOHQ' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

address = 'Rua Joao Pio Duarte Silva, n 250, Florianopolis, Santa Catarina, Brazil'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

Your credentails:
CLIENT_ID: 00RRGR34RRRRSAZBIIEHKUQSDC0OIHWY0X3LKNSWA3CBCMDY
CLIENT_SECRET:VQEO5QPVRG0KORW4HJKNOAMMQQZVRGBQAK145E30WKCRKOHQ
-27.6005244 -48.514517


In [36]:
search_query = 'Café'
radius = 500
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ead9a25d03993001b73fb0a'},
 'response': {'venues': [{'id': '57f3f87e498ed28dbb288cd0',
    'name': 'Arte Café Padaria Buffet a Kg',
    'location': {'lat': -27.600246,
     'lng': -48.515293,
     'labeledLatLngs': [{'label': 'display',
       'lat': -27.600246,
       'lng': -48.515293}],
     'distance': 82,
     'postalCode': '88037-001',
     'cc': 'BR',
     'city': 'Florianópolis',
     'state': 'SC',
     'country': 'Brasil',
     'formattedAddress': ['Florianópolis, SC', '88037-001', 'Brasil']},
    'categories': [{'id': '4bf58dd8d48988d16a941735',
      'name': 'Bakery',
      'pluralName': 'Bakeries',
      'shortName': 'Bakery',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/bakery_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1588435592',
    'hasPerk': False},
   {'id': '5bb13aad1c675b002c935e67',
    'name': 'Carta’s Café',
    'location': {'address': 'Rua Ana Maria Nunes, 15',
     

In [37]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d16a941735', 'name': 'B...",False,57f3f87e498ed28dbb288cd0,NaN,BR,Florianópolis,Brasil,82,"[Florianópolis, SC, 88037-001, Brasil]","[{'label': 'display', 'lat': -27.600246, 'lng'...",-27.600246,-48.515293,NaN,88037-001,SC,Arte Café Padaria Buffet a Kg,v-1588435592
1,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",False,5bb13aad1c675b002c935e67,"Rua Ana Maria Nunes, 15",BR,Florianópolis,Brasil,142,"[Rua Ana Maria Nunes, 15, Florianópolis, SC, 8...","[{'label': 'display', 'lat': -27.600678, 'lng'...",-27.600678,-48.513084,NaN,88037-020,SC,Carta’s Café,v-1588435592
2,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",False,4eb42d48b8f786e274a624c2,"R. João Pio Duarte Silva, S/N",BR,Florianópolis,Brasil,162,"[R. João Pio Duarte Silva, S/N, Florianópolis,...","[{'label': 'display', 'lat': -27.6001819913326...",-27.600182,-48.516115,NaN,88037-000,SC,Café Mané,v-1588435592
3,"[{'id': '4bf58dd8d48988d116941735', 'name': 'B...",False,4cdf388b94c3b60c16906aea,"R. João Pio Duarte Silva, 94",BR,Florianópolis,Brasil,117,"[R. João Pio Duarte Silva, 94, Florianópolis, ...","[{'label': 'display', 'lat': -27.6002061555286...",-27.600206,-48.515648,Córrego Grande,88037-000,SC,Café Del Sur,v-1588435592
